In [1]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import tinycudann as tcnn
import torch.nn as nn

from nemo.nemo import Nemo
from nemo.util import grid_2d
from nemo.siren import Siren
from nemo.plotting import plot_surface, plot_path_3d
from nemo.dynamics import diff_flatness_siren

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2

In [3]:
# Get data to fit
nemo = Nemo()
nemo.load_weights('../models/AirSimMountains/AirSimMountains_encs.pth', '../models/AirSimMountains/AirSimMountains_mlp.pth')

N = 512
bounds = np.array([-0.75, 0.45, -0.6, 0.6])
xy, grid = grid_2d(N, bounds)
xy = xy.to(device)
z = nemo.get_heights(xy)

# SIREN

In [2]:
siren = Siren(in_features=2, out_features=1, hidden_features=256,
                hidden_layers=3, outermost_linear=True).to(device)

## Train

Siren weights are torch.float32

In [ ]:
next(siren.named_parameters())[1].dtype

In [ ]:
# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(siren.parameters(), lr=1e-5)

# Convert data from half to float
xy = xy.to(torch.float32).to(device)
z = z.to(torch.float32).to(device)

# Train the network
for step in range(1000):
    # Forward pass
    pred = siren(xy)

    # Compute loss
    loss = criterion(pred, z)

    # Backward pass
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

    # Print loss every 500 steps
    if step % 500 == 0:
        print(f"Step {step}, Loss {loss.item()}")

In [ ]:
# Sample the Siren network to get the predicted elevation
with torch.no_grad():
    pred = siren(xy)

# Plot the predictions
z_grid = pred.reshape(N, N).detach().cpu().numpy()
x_grid = grid[:,:,0].detach().cpu().numpy()
y_grid = grid[:,:,1].detach().cpu().numpy()

fig = plot_surface(x_grid, y_grid, z_grid, no_axes=False, showscale=False)
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
torch.save(siren.state_dict(), '../models/airsim_mountains_siren.pt')

## Load

In [3]:
siren.load_state_dict(torch.load('../models/airsim_mountains_siren.pt'))

N = 512
bounds = np.array([-0.75, 0.45, -0.6, 0.6])
xy, grid = grid_2d(N, bounds)
xy = xy.to(device)
z = siren(xy)

x_grid = grid[:,:,0].detach().cpu().numpy()
y_grid = grid[:,:,1].detach().cpu().numpy()
z_grid = z.reshape(N, N).detach().cpu().numpy()

In [ ]:
fig = plot_surface(x_grid, y_grid, z_grid, no_axes=False, showscale=False)
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# Plot gradients
_, grad = siren.forward_with_grad(xy.clone().requires_grad_(True))

grad = grad.reshape(N, N, 2).detach().cpu().numpy()
grad_x = grad[:,:,0]
grad_y = grad[:,:,1]

fig = make_subplots(rows=1, cols=2, subplot_titles=('X Gradient', 'Y Gradient'), horizontal_spacing=0.15)
fig.add_trace(go.Heatmap(z=grad_x, colorbar=dict(len=1.05, x=0.44, y=0.5)), row=1, col=1)
fig.add_trace(go.Heatmap(z=grad_y, colorbar=dict(len=1.05, x=1.01, y=0.5)), row=1, col=2)
fig.update_layout(width=1300, height=600, scene_aspectmode='data')
fig.show()

# Path planning

In [7]:
from nemo.global_planner import AStarGradPlanner

In [8]:
nemo_start = np.array([0.404, 0.156, -0.613])
nemo_end = np.array([-0.252, -0.228, -0.477])

In [9]:
# Initialize the planner with scaled heightmap
astar_heights = 1e4 * (z_grid + 1.0).reshape(N, N)
gp = AStarGradPlanner(astar_heights, bounds)

# Start and end positions for path
start = tuple(nemo_start[:2])
end = tuple(nemo_end[:2])

# Compute path
path_xy = gp.spatial_plan(start, end)
path_xy_torch = torch.tensor(path_xy, dtype=torch.float32, device=device)
# Get heights along path
path_zs = siren(path_xy_torch)  

# Save path as torch tensor
astar_path = torch.cat((path_xy_torch, path_zs), dim=1)

init_xy = path_xy_torch
init_xy = init_xy.to(torch.float32).to(device)

init_z = siren(init_xy)
init_path = torch.cat((init_xy, init_z), dim=1)

In [ ]:
fig = go.Figure()
fig = plot_surface(x_grid, y_grid, z_grid, fig=fig, no_axes=True)
fig = plot_path_3d(x=init_path[:,0].detach().cpu().numpy(),
                        y=init_path[:,1].detach().cpu().numpy(),
                        z=init_path[:,2].detach().cpu().numpy(), color='red', fig=fig)
fig.show()

In [34]:
path_start = init_xy[0]
path_end = init_xy[-1]
path_opt = init_xy[1:-1].clone().detach().requires_grad_(True)  # portion of the path to optimize
path = torch.cat((path_start[None], path_opt, path_end[None]), dim=0)  # full path

In [35]:
u = diff_flatness_siren(path, siren, dt=1.0)

## Test gradient flow

In [47]:
dt = 1.0
xy = path

x = xy[:, 0]
y = xy[:, 1]
epsilon = torch.tensor(1e-5, device=device, requires_grad=True)
xdot = torch.hstack((epsilon, torch.diff(x) / dt))
ydot = torch.hstack((epsilon, torch.diff(y) / dt))
xddot = torch.hstack((epsilon, torch.diff(xdot) / dt))
yddot = torch.hstack((epsilon, torch.diff(ydot) / dt))
v = torch.sqrt(xdot**2 + ydot**2)
theta = torch.arctan2(ydot, xdot)

_, grad = siren.forward_with_grad(xy.clone().requires_grad_(True))
psi = torch.atan2(grad[:,1], grad[:,0])
alpha = torch.atan(grad.norm(dim=1))

phi = alpha * torch.cos(theta - psi)
g_eff = 9.81 * torch.sin(phi)

In [48]:
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer

u = cp.Variable(2)
J = cp.Parameter((2, 2))
b = cp.Parameter(2)
objective = cp.Minimize(cp.norm(J @ u - b))
problem = cp.Problem(objective)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[J, b], variables=[u])

In [ ]:
i = 0
J_tch = torch.tensor([[torch.cos(theta[i]), -v[i] * torch.sin(theta[i])],
                        [torch.sin(theta[i]), v[i] * torch.cos(theta[i])]], device=device, requires_grad=True)
b_tch = torch.tensor([xddot[i] + g_eff[i] * torch.cos(theta[i]),
                        yddot[i] + g_eff[i] * torch.sin(theta[i])], device=device, requires_grad=True)

solution, = cvxpylayer(J_tch, b_tch)

solution.sum().backward()
solution

In [50]:
u = torch.zeros(len(x), 2)
for i in range(len(x)):
    J_tch = torch.stack([torch.cos(theta[i]), -v[i] * torch.sin(theta[i]),
                     torch.sin(theta[i]), v[i] * torch.cos(theta[i])]).reshape(2, 2)
    b_tch = torch.stack([xddot[i] + g_eff[i] * torch.cos(theta[i]),
                     yddot[i] + g_eff[i] * torch.sin(theta[i])])
    u[i], = cvxpylayer(J_tch, b_tch)

    # c_test = torch.sum(u[i])
    # c_test.backward(retain_graph=True)  # Use retain_graph to avoid re-building
    # print(path_opt.grad)

In [ ]:
c = torch.sum(u**2)
c.backward()
print(path_opt.grad)

## Path optimization

TODO: 
* in addition to penalizing control input, also penalize change in control inputs
* enforce control input limits
* fix initial heading

In [ ]:
path_xy = init_xy
path_start = path_xy[0]
path_end = path_xy[-1]
path_opt = path_xy[1:-1].clone().detach().requires_grad_(True)  # portion of the path to optimize

# Optimize path
opt = torch.optim.Adam([path_opt], lr=1e-3)

a_cost_coeff = 1.0
omega_cost_coeff = 1000.0

for it in range(100):
    opt.zero_grad()
    path = torch.cat((path_start[None], path_opt, path_end[None]), dim=0)
    u = diff_flatness_siren(path, siren, dt=1.0)

    c = a_cost_coeff * torch.sum(u[:,0]**2) + omega_cost_coeff * torch.sum(u[:,1]**2)
    
    c.backward()
    opt.step()
    if it % 10 == 0:
        print(f'it: {it},  Cost: {c.item()}')

print(f'Finished optimization - final cost: {c.item()}')

# Compute final heights and get full 3D path
path_zs = siren(path)
path_3d = torch.cat((path, path_zs), dim=1)

In [ ]:
a = u[:,0]
omega = u[:,1]

# Plot over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(a)), y=a.detach().cpu().numpy(), mode='lines', name='a'))
fig.add_trace(go.Scatter(x=np.arange(len(omega)), y=omega.detach().cpu().numpy(), mode='lines', name='omega'))
fig.update_layout(title='Control inputs over time', xaxis_title='Time', yaxis_title='Control Input')
fig.show()

In [ ]:
fig = go.Figure()
fig = plot_surface(x_grid, y_grid, z_grid, fig=fig, no_axes=True)
fig = plot_path_3d(x=path_3d[:,0].detach().cpu().numpy(), 
                        y=path_3d[:,1].detach().cpu().numpy(), 
                        z=path_3d[:,2].detach().cpu().numpy(), color='orange', fig=fig)
fig = plot_path_3d(x=init_path[:,0].detach().cpu().numpy(),
                        y=init_path[:,1].detach().cpu().numpy(),
                        z=init_path[:,2].detach().cpu().numpy(), color='red', fig=fig)
fig.show()

# Cooper

In [12]:
import cooper
from nemo.dynamics import forward_dynamics

In [17]:
# Forward dynamics

# x, y, theta, v
x_start = torch.tensor([0.404, 0.156], device=device)
theta_start = torch.tensor(0.0, device=device)
v_start = torch.tensor(0.0, device=device)
x_goal = torch.tensor([-0.252, -0.228], device=device)
v_goal = torch.tensor(0.0, device=device)
# goal theta is unconstrained

# Initial control inputs
num_steps = 100
u = torch.zeros(num_steps, 2, device=device)
init_state = torch.tensor([0.404, 0.156, 0.0, 0.0], device=device)

traj = forward_dynamics(init_state, u, siren, dt=1.0)

resulting_path_2d = traj[:,:2].to(device)
resulting_path_z = siren(resulting_path_2d)

In [ ]:
fig = go.Figure()
fig = plot_surface(x_grid, y_grid, z_grid, fig=fig, no_axes=True)
fig = plot_path_3d(x=resulting_path_2d[:,0].detach().cpu().numpy(), 
                        y=resulting_path_2d[:,1].detach().cpu().numpy(), 
                        z=resulting_path_z.detach().cpu().numpy(), color='orange', fig=fig)
fig.show()

In [ ]:
class MaximumEntropy(cooper.ConstrainedMinimizationProblem):
    def __init__(self, mean_constraint):
        self.mean_constraint = mean_constraint
        super().__init__(is_constrained=True)

    def closure(self, probs):
        # Verify domain of definition of the functions
        assert torch.all(probs >= 0)

        # Negative signed removed since we want to *maximize* the entropy
        entropy = torch.sum(probs * torch.log(probs))

        # Entries of p >= 0 (equiv. -p <= 0)
        ineq_defect = -probs

        # Equality constraints for proper normalization and mean constraint
        mean = torch.sum(torch.tensor(range(1, len(probs) + 1)) * probs)
        eq_defect = torch.stack([torch.sum(probs) - 1, mean - self.mean_constraint])

        return cooper.CMPState(loss=entropy, eq_defect=eq_defect, ineq_defect=ineq_defect)

# Define the problem and formulation
cmp = MaximumEntropy(mean_constraint=4.5)
formulation = cooper.LagrangianFormulation(cmp)

# Define the primal parameters and optimizer
probs = torch.nn.Parameter(torch.rand(6)) # Use a 6-sided die
primal_optimizer = cooper.optim.ExtraSGD([probs], lr=3e-2, momentum=0.7)

# Define the dual optimizer. Note that this optimizer has NOT been fully instantiated
# yet. Cooper takes care of this, once it has initialized the formulation state.
dual_optimizer = cooper.optim.partial_optimizer(cooper.optim.ExtraSGD, lr=9e-3, momentum=0.7)

# Wrap the formulation and both optimizers inside a ConstrainedOptimizer
coop = cooper.ConstrainedOptimizer(formulation, primal_optimizer, dual_optimizer)

# Here is the actual training loop.
# The steps follow closely the `loss -> backward -> step` Pytorch workflow.
for iter_num in range(5000):
    coop.zero_grad()
    lagrangian = formulation.composite_objective(cmp.closure, probs)
    formulation.custom_backward(lagrangian)
    coop.step(cmp.closure, probs)